# Reproduction Table 1, CNN
In this notebook the CNN result of table 1 is reproduced using PyTorch.




In [ ]:
 # run this cell to download the right packages (only needed once)
!python --version

!pip install cifar10
!pip install imageio numpy scipy    
!pip install git+https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks
!pip install torchinfo


Python 3.9.16
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks to /tmp/pip-req-build-f2d4i7a4
  Running command git clone --filter=blob:none --quiet https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks /tmp/pip-req-build-f2d4i7a4
  Resolved https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks to commit 28caa7cde240e354fd7b87280450fd233cd494c3
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torchvision

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary
from torch.utils.data import DataLoader
from torchinfo import summary as summary_info
from torchvision.transforms import transforms
from torchvision import datasets

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
%%time

# import and download the CIFAR10 dataset
batch_size = 32

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5,.5,.5),(.5,.5,.5))])                                                  

transform_train = transforms.Compose([transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)


classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified
CPU times: user 1.35 s, sys: 301 ms, total: 1.65 s
Wall time: 1.69 s


In [ ]:
class ConvNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_features, kernel_size):
        super(ConvNet, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels, hidden_channels[0], kernel_size)
        self.conv_2 = nn.Conv2d(hidden_channels[0], hidden_channels[1], kernel_size)

        self.pool_1 = nn.MaxPool2d(2, 2)
        self.dropout_1 = nn.Dropout(0.25)

        self.conv_3 = nn.Conv2d(hidden_channels[1], hidden_channels[2], kernel_size)
        self.conv_4 = nn.Conv2d(hidden_channels[2], hidden_channels[3], kernel_size)

        self.pool_2 = nn.MaxPool2d(2, 2)
        self.dropout_2 = nn.Dropout(0.25)

        self.fc_1 = nn.Linear(4608, 512)
        self.fc_2 = nn.Linear(512, out_features)

        self.dropout_3 = nn.Dropout(0.5)
        self.sm = nn.Softmax(dim=1)

    def forward(self,x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool_1(x)
        x = self.dropout_1(x)

        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))
        x = self.pool_2(x)
        x = self.dropout_2(x)

        x = torch.flatten(x, start_dim=1) 

        x = F.relu(self.fc_1(x))
        x = self.dropout_3(x)
        x = self.fc_2(x)
        x = self.sm(x)

        return x


# Model parameters
in_channels = 3
hidden_channels = [64, 128, 256, 512]
out_features = 10
kernel_size = 4

cnn = ConvNet(in_channels, hidden_channels, out_features, kernel_size)
cnn = cnn.cuda()
print("Number of trainable parameters: ", sum(p.numel() for p in cnn.parameters() if p.requires_grad))

# print model summary
summary(cnn, input_size=(in_channels, 32, 32), batch_size=batch_size, device=device.type)


Number of trainable parameters:  5121482
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [32, 64, 29, 29]           3,136
            Conv2d-2          [32, 128, 26, 26]         131,200
         MaxPool2d-3          [32, 128, 13, 13]               0
           Dropout-4          [32, 128, 13, 13]               0
            Conv2d-5          [32, 256, 10, 10]         524,544
            Conv2d-6            [32, 512, 7, 7]       2,097,664
         MaxPool2d-7            [32, 512, 3, 3]               0
           Dropout-8            [32, 512, 3, 3]               0
            Linear-9                  [32, 512]       2,359,808
          Dropout-10                  [32, 512]               0
           Linear-11                   [32, 10]           5,130
          Softmax-12                   [32, 10]               0
Total params: 5,121,482
Trainable params: 5,121,482
Non-traina

In [ ]:
%%time
num_epochs = 80
learning_rate = 0.0001
learning_rate_decay = 1e-6

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(cnn.parameters(),lr=learning_rate, weight_decay=learning_rate_decay)

n_total_step = len(train_loader)
        
for epoch in range(num_epochs):
  for i,(images, labels) in enumerate(train_loader):
      images = images.cuda()
      labels = labels.cuda()
      outputs = cnn(images)
      loss = criterion(outputs, labels)
      
      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
  print(f"Epoch [{epoch+1}/{num_epochs}], Last loss: {loss.item():.4f}")
    
print("Finished training")    

Epoch [1/80], Last loss: 2.0785
Epoch [2/80], Last loss: 2.2880
Epoch [3/80], Last loss: 2.0752
Epoch [4/80], Last loss: 1.8501
Epoch [5/80], Last loss: 1.8796
Epoch [6/80], Last loss: 1.8211
Epoch [7/80], Last loss: 1.7624
Epoch [8/80], Last loss: 1.8702
Epoch [9/80], Last loss: 1.8104
Epoch [10/80], Last loss: 1.7699
Epoch [11/80], Last loss: 1.8345
Epoch [12/80], Last loss: 1.8089
Epoch [13/80], Last loss: 1.7506
Epoch [14/80], Last loss: 1.8241
Epoch [15/80], Last loss: 1.6014
Epoch [16/80], Last loss: 1.8853
Epoch [17/80], Last loss: 1.4983
Epoch [18/80], Last loss: 1.6422
Epoch [19/80], Last loss: 1.8272
Epoch [20/80], Last loss: 1.5766
Epoch [21/80], Last loss: 1.6398
Epoch [22/80], Last loss: 1.9576
Epoch [23/80], Last loss: 1.6632
Epoch [24/80], Last loss: 1.6935
Epoch [25/80], Last loss: 1.4854
Epoch [26/80], Last loss: 1.6618
Epoch [27/80], Last loss: 1.8221
Epoch [28/80], Last loss: 1.8102
Epoch [29/80], Last loss: 1.5862
Epoch [30/80], Last loss: 1.6905
Epoch [31/80], Last

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0

    n_class_correct = [0 for i in range(10)]
    n_class_sample = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = cnn(images)
       

        _, predicted = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(labels.size(0)):
            label = labels[i]
            pred = predicted[i]
            if(label==pred):
                n_class_correct[label] += 1
            n_class_sample[label] += 1

    acc = 100 * n_correct / n_samples
    print(f"Accuracy of the network: {acc}%")

    for i in range(10):
        acc = 100 * n_class_correct[i]/ n_class_sample[i]
        print(f"Accuracy of {classes[i]}: {acc} %")

Accuracy of the network: 75.83%
Accuracy of plane: 81.0 %
Accuracy of car: 86.8 %
Accuracy of bird: 71.9 %
Accuracy of cat: 58.7 %
Accuracy of deer: 77.9 %
Accuracy of dog: 55.1 %
Accuracy of frog: 82.2 %
Accuracy of horse: 77.6 %
Accuracy of ship: 84.9 %
Accuracy of truck: 82.2 %
